In [9]:
import nbformat

# Read your notebook (assuming version 4 for example purposes)
nb = nbformat.read("ml_project.ipynb", as_version=4)

# Normalize the notebook to add missing id fields and other updates
nbformat.validator.validate(nb)

# Write the normalized notebook back to a file
nbformat.write(nb, "ml_project_normalized.ipynb")

In [10]:
%run ml_project_normalized.ipynb

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
dropped columns
cleaned intake time
cleaned intake condition
cleaned age and sex
cleaned color
cleaned breed
dropped columns


/tmp/ipykernel_898769/1199118304.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dt_series = pd.to_datetime(df['intake_time'], errors='coerce')


cleaned intake time
cleaned intake condition
cleaned age and sex
cleaned color
cleaned breed
Done running ml_project.ipynb.


In [11]:
#kog_reg
nb = nbformat.read("log_reg.ipynb", as_version=4)
nbformat.validator.validate(nb)
nbformat.write(nb, "log_reg_normalized.ipynb")

# Random Frest
nb = nbformat.read("random_forest.ipynb", as_version=4)
nbformat.validator.validate(nb)
nbformat.write(nb, "random_forest_normalized.ipynb")

# X Random Trees
nb = nbformat.read("x_random_trees.ipynb", as_version=4)
nbformat.validator.validate(nb)
nbformat.write(nb, "x_random_trees_normalized.ipynb")

In [12]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import balanced_accuracy_score
from imblearn.pipeline import Pipeline as ImbPipeline  # Important for SMOTE compatibility
from imblearn.over_sampling import SMOTENC

In [13]:
df_train = bucket_seasons(df_train)
df_train = time_as_cyclical(df_train)
df_train = df_train.drop(columns=['month_sin', 'month_cos'])

df_test = bucket_seasons(df_test)
df_test = time_as_cyclical(df_test)
df_test = df_test.drop(columns=['month_sin', 'month_cos'])

df_train = df_train.drop(columns=['size'])
df_test = df_test.drop(columns=['size'])
df_train.head()

,intake_type,intake_condition,animal_type,sex_upon_intake,age_upon_intake,breed,intake_year,primary_color,is_mix,outcome_type,season,hour_sin,hour_cos
0,Stray,Normal / Behavior,Dog,Spayed Female,96.0,2,2015,white,0,Return to Owner,Summer,1.224647e-16,-1.000000e+00
1,Stray,Normal / Behavior,Dog,Intact Male,11.0,7,2016,sable,1,Return to Owner,Spring,-1.000000e+00,-1.836970e-16
2,Public Assist,Normal / Behavior,Cat,Neutered Male,24.0,Common,2022,orange,0,Transfer,Spring,0.000000e+00,1.000000e+00
3,Owner Surrender,Normal / Behavior,Dog,Neutered Male,24.0,2,2017,chocolate,1,Return to Owner,Winter,1.224647e-16,-1.000000e+00
4,Public Assist,Normal / Behavior,Dog,Neutered Male,72.0,3,2019,black,1,Return to Owner,Spring,7.071068e-01,-7.071068e-01


In [14]:
X = df_train.drop(columns=['outcome_type'])
y = df_train['outcome_type']

X_train, X_test, y_train, y_test = train_test_split(X,y , test_size=0.20, random_state=42)

In [15]:
# rerun when i make changes
# %run log_reg.ipynb
#%run random_forest.ipynb

X.head()

,intake_type,intake_condition,animal_type,sex_upon_intake,age_upon_intake,breed,intake_year,primary_color,is_mix,season,hour_sin,hour_cos
0,Stray,Normal / Behavior,Dog,Spayed Female,96.0,2,2015,white,0,Summer,1.224647e-16,-1.000000e+00
1,Stray,Normal / Behavior,Dog,Intact Male,11.0,7,2016,sable,1,Spring,-1.000000e+00,-1.836970e-16
2,Public Assist,Normal / Behavior,Cat,Neutered Male,24.0,Common,2022,orange,0,Spring,0.000000e+00,1.000000e+00
3,Owner Surrender,Normal / Behavior,Dog,Neutered Male,24.0,2,2017,chocolate,1,Winter,1.224647e-16,-1.000000e+00
4,Public Assist,Normal / Behavior,Dog,Neutered Male,72.0,3,2019,black,1,Spring,7.071068e-01,-7.071068e-01


In [16]:
def train_voting_classifier(X_train, y_train, X_test, models, voting='soft'):
    """
    Trains a VotingClassifier with the given models and evaluates it on the test data.
    
    Parameters:
    - X_train: Training features
    - y_train: Training labels
    - X_test: Test features
    - models: List of tuples (model_name, model_instance)
    - voting: 'hard' or 'soft' for the type of voting to be used (default is 'soft')
    
    Returns:
    - trained_model: Trained VotingClassifier
    - test_predictions: Predictions on the test set
    """
    # Initialize the VotingClassifier with the given models
    voting_clf = VotingClassifier(estimators=models, voting=voting, n_jobs=-1)
    
    # Fit the model
    voting_clf.fit(X_train, y_train)
    
    # Make predictions
    test_predictions = voting_clf.predict(X_test)
    
    # Evaluate the model
    accuracy = balanced_accuracy_score(y_train, voting_clf.predict(X_train))
    print(f"Training Accuracy: {accuracy:.4f}")
    
    return voting_clf, test_predictions

# needs CV loop after testing

In [17]:
# Example usage:
models = [
    ('logreg', training_logreg_classifier()), #logistic regression
    ('rf', train_RF_classifier()), #random forest
    ('xrt', train_XRT_classifier()) #x random trees
]

# Assuming X_train, y_train, X_test are already defined in your environment
voting_model, test_predictions = train_voting_classifier(X_train, y_train, X_test, models, voting='soft')

NameError: name 'training_logreg_classifier' is not defined

In [ ]:
# assuming these are pipelines or models you've already trained/tuned
voting_clf = VotingClassifier(
    estimators=[
        ('xgb', xgb_model),      # best XGBoost model
        ('cat', catboost_model), # best CatBoost model
        ('logreg', log_reg_model),
        ('rf', rf_model),
        ('nb', nb_model)
    ],
    voting='soft',
  #  weights=[3, 2.5, 2, 1.5, 1],  # optional: weight more accurate models
    n_jobs=-1
)

voting_clf.fit(X_train, y_train)
y_pred = voting_clf.predict(X_test)

# Evaluate
from sklearn.metrics import balanced_accuracy_score
print("Ensemble Balanced Accuracy:", balanced_accuracy_score(y_test, y_pred))
